In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import date
from bs4 import BeautifulSoup, Comment

In [2]:
today = date.today()

# dd/mm/YY
current_year = today.strftime("%Y")

last_year = int(current_year) - 1

In [3]:
last_five_years = []
for i in range(0,5):
    last_five_years.append(int(current_year) - i)

In [4]:
# Create a list to help create a dataframe from batter statistics data
nba_stats = []

# Create a loop to create a dataframe from Basketball Reference tables
for year in last_five_years:
    
    # input URL and use BeautifulSoup to parse through the page
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_totals.html'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    # Grab the table element that has NBA player statistics
    table = soup.select_one('#div_totals_stats') 

    # Grab data from table and put it into the list created above
    for tr in table.select('tr:has(td)'):
        tds = [td.get_text(strip=True) for td in tr.select('td')]
        tds.append(year)
        nba_stats.append(tds)

In [5]:
# Create dataframe for batter statistics
nba_stats_df = pd.DataFrame(nba_stats)

# Create an empty list to store dataframe header information
header_list = []

# Grab the table header information to use as column headers in our dataframe
for tr in table.select('tr:has(th)'):
    ths = [th.get_text(strip=True) for th in tr.select('th')]
    header_list.append(ths)

# For loop returns a list of lists, and we only need the first list 
df_headers = header_list[0]

# Remove the first item from our headers list, it is the index header that we do not need
df_headers.remove('Rk')
df_headers.append("Year")

# Set column headers equal to our list
nba_stats_df.columns = df_headers

In [6]:
# Change types of columns to numeric for columns with number values
nba_stats_df[['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']] = nba_stats_df[['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']].apply(pd.to_numeric)

# Drop all players with NaN games to remove null values
nba_stats_df.dropna(subset=['G'], axis = 0 , inplace= True)

# Remove any players with fewer than 27 games played (1/3rd of the season)
filtered_nba_stats_df = nba_stats_df[nba_stats_df['G'] >= 27]

# points, rebounds, assists, steals, blocks, threes, field-goal percentage, and free-throw percentage

# Select the columns we want for our NBA player analysis
yahoo_nba_df = filtered_nba_stats_df[['Year','Player','Age','Pos','Team','G','FG%','3P','FT%','TRB','AST','STL','BLK','PTS']]


In [7]:
# Sort by index to prepare to drop duplicates
yahoo_nba_df = yahoo_nba_df.sort_index()

# Drop duplicate entries of Player Name and Year
# This is to eliminate partial season data for players who played for 2+ teams in one season
yahoo_nba_df = yahoo_nba_df.drop_duplicates(subset=['Year', 'Player'])

In [8]:
# Sort data by name alphabetically, then by year in descending order
final_yahoo_nba_df = yahoo_nba_df.sort_values(['Year','Player'], ascending=[True, True])
final_yahoo_nba_df

,Year,Player,Age,Pos,Team,G,FG%,3P,FT%,TRB,AST,STL,BLK,PTS
3027,2020,Aaron Gordon,24.0,PF,ORL,62.0,0.437,73.0,0.674,475.0,228.0,51.0,39.0,894.0
3105,2020,Aaron Holiday,23.0,PG,IND,66.0,0.414,87.0,0.851,156.0,225.0,55.0,16.0,627.0
3245,2020,Abdel Nader,26.0,SF,OKC,55.0,0.468,48.0,0.773,100.0,38.0,23.0,20.0,345.0
3417,2020,Admiral Schofield,22.0,PF,WAS,33.0,0.380,19.0,0.667,47.0,15.0,8.0,4.0,99.0
3048,2020,Al Horford,33.0,C,PHI,67.0,0.450,99.0,0.763,456.0,270.0,52.0,61.0,798.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,2024,Yuta Watanabe,29.0,SF,2TM,34.0,0.353,25.0,0.588,54.0,15.0,12.0,7.0,117.0
167,2024,Zach Collins,26.0,C,SAS,69.0,0.484,58.0,0.753,370.0,194.0,34.0,52.0,776.0
451,2024,Zeke Nnaji,23.0,PF,DEN,58.0,0.463,6.0,0.677,126.0,32.0,15.0,38.0,186.0
299,2024,Ziaire Williams,22.0,SF,MEM,51.0,0.397,58.0,0.827,180.0,75.0,36.0,9.0,420.0


In [9]:
# Create a new dataframe for stats percentile calculations
percentile_df = pd.DataFrame(columns = ['Year', 'Player', 'Age', 'Pos', 'Team', '3P_Percentile', 'FG%_Percentile', 'TRB_Percentile', 'FT%_Percentile', 'AST_Percentile', 'STL_Percentile', 'BLK_Percentile', 'PTS_Percentile'])

# Carry over your non-numeric columnns from final_yahoo_nba_df
percentile_df['Year'] = final_yahoo_nba_df['Year']
percentile_df['Player'] = final_yahoo_nba_df['Player']
percentile_df['Age'] = final_yahoo_nba_df['Age']
percentile_df['Pos'] = final_yahoo_nba_df['Pos']
percentile_df['Team'] = final_yahoo_nba_df['Team']

# Calculate the percentile rank for each player in each season, seperately, then add all the seasons in one dataframe
for year in last_five_years:
    year_df = final_yahoo_nba_df.loc[final_yahoo_nba_df['Year'] == year]
    year_df['3P_Percentile'] = year_df['3P'].rank(pct=True)
    year_df['FG%_Percentile'] = year_df['FG%'].rank(pct=True)
    year_df['TRB_Percentile'] = year_df['TRB'].rank(pct=True)
    year_df['FT%_Percentile'] = year_df['FT%'].rank(pct=True)
    year_df['AST_Percentile'] = year_df['AST'].rank(pct=True)
    year_df['STL_Percentile'] = year_df['STL'].rank(pct=True)
    year_df['BLK_Percentile'] = year_df['BLK'].rank(pct=True)
    year_df['PTS_Percentile'] = year_df['PTS'].rank(pct=True)
    year_df.sort_values('Player', ascending=True)

    # Each of the seasons are added back to the percentile dataframe
    percentile_df = percentile_df.append(year_df, ignore_index=True)



/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_93652/2406144189.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df['3P_Percentile'] = year_df['3P'].rank(pct=True)
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_93652/2406144189.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df['FG%_Percentile'] = year_df['FG%'].rank(pct=True)
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_93652/2406144189.py:16: SettingWithCopyWarning: 
A value is trying to be set on a

In [10]:
# Clean up the percentile dataframe, drop NaNs and remove unnecessary columns
percentile_df = percentile_df.dropna()
percentile_df = percentile_df.drop(['Team','G','FG%','3P','FT%','TRB','AST','STL','BLK','PTS'], axis = 1)

# Add a rank column that adds the percentiles from each category
# Average out percentile categories for FG% and FT% with PTS, as we might still want a player with somewhat  
## lower FG% and FT% if they score a lot of points
percentile_df['Rank'] = (percentile_df['3P_Percentile'] + ((percentile_df['FG%_Percentile'] + percentile_df['PTS_Percentile']) / 2) + percentile_df['TRB_Percentile'] + ((percentile_df['FT%_Percentile'] + percentile_df['PTS_Percentile']) / 2) + percentile_df['AST_Percentile'] + percentile_df['STL_Percentile'] + percentile_df['BLK_Percentile'] + percentile_df['PTS_Percentile'])


In [11]:
# Create a list of each unique player we have in our dataframe
player_list = percentile_df.Player.unique().tolist()

# Create a new dataframe for combined, averaged percentiles over the past 5 seasons
new_df = pd.DataFrame(columns = ['Player', 'Rank', 'Trend', 'Pos', 'Years', '3P_Percentile', 'FG%_Percentile', 'TRB_Percentile', 'FT%_Percentile', 'AST_Percentile', 'STL_Percentile', 'BLK_Percentile', 'PTS_Percentile'])

# Update new dataframe with unique player list
new_df['Player'] = player_list

# Create a list for each percentile stat category for upcoming loop
player_trends = []
average_3P = []
average_FG = []
average_TRB = []
average_FT = []
average_AST = []
average_STL = []
average_BLK = []
average_PTS = []
average_Rank = []
year_count = []
pos = []

# Loop through each player, locate their percentile stats for each season, average them out
for player in player_list:
    player_df = percentile_df.loc[percentile_df['Player'] == player]
    
    # We want to find the slope of the line of best fit for each player's overall ranking each season
    x = np.array(player_df['Year'], dtype = float)
    y = np.array(player_df['Rank'], dtype = float)
    slope, intercept = np.polyfit(x, y, 1)
    player_trends.append(slope)
    
    # Find average of each player's percentiles from previous 5 seasons
    average_3P.append(sum(player_df['3P_Percentile']) / len(player_df['3P_Percentile']))
    average_FG.append(sum(player_df['FG%_Percentile']) / len(player_df['FG%_Percentile']))
    average_TRB.append(sum(player_df['TRB_Percentile']) / len(player_df['TRB_Percentile']))
    average_FT.append(sum(player_df['FT%_Percentile']) / len(player_df['FT%_Percentile']))
    average_AST.append(sum(player_df['AST_Percentile']) / len(player_df['AST_Percentile']))
    average_STL.append(sum(player_df['STL_Percentile']) / len(player_df['STL_Percentile']))
    average_BLK.append(sum(player_df['BLK_Percentile']) / len(player_df['BLK_Percentile']))
    average_PTS.append(sum(player_df['PTS_Percentile']) / len(player_df['PTS_Percentile']))
    average_Rank.append(sum(player_df['Rank']) / len(player_df['Rank']))
    year_count.append(len(x))
    
    # Keep player positions for reference purposes during the draft
    pos.append(player_df['Pos'].unique())

# Update new dataframe with the list data from each stat
new_df['Pos'] = pos
new_df['Trend'] = player_trends
new_df['3P_Percentile'] = average_3P
new_df['FG%_Percentile'] = average_FG
new_df['TRB_Percentile'] = average_TRB
new_df['FT%_Percentile'] = average_FT
new_df['AST_Percentile'] = average_AST
new_df['STL_Percentile'] = average_STL
new_df['BLK_Percentile'] = average_BLK
new_df['PTS_Percentile'] = average_PTS
new_df['Rank'] = average_Rank

# Keep track of how many seasons are being considered, so we know how reliable the data is
new_df['Years'] = year_count



/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

In [12]:
# Create a weighted rank column by adding trend data to the rank data and account for number of seasons played
# Basically, if you played all 5 seasons, your trend stat is added directly
# If you played fewer than all 5 seasons, your trend stat is reduced depending on how few seasons you played
new_df['Weighted Rank'] = (new_df['Rank'] + ((new_df['Trend'] * (new_df['Years'] - 1) / 4)))

# shift column 'Weighted Rank' to first position
first_column = new_df.pop('Weighted Rank')
  
# insert column using insert(position,column_name,first_column) function
new_df.insert(1, 'Weighted Rank', first_column)

new_df = new_df.sort_values('Weighted Rank', ascending = False)
new_df.head(50)

,Player,Weighted Rank,Rank,Trend,Pos,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile
309,Nikola Jokić,7.229185,7.225615,0.003570,[C],5,0.641431,0.905583,0.992384,0.702533,0.992733,0.960128,0.877943,0.978469
190,Jayson Tatum,7.200213,7.170453,0.029761,"[PF, SF]",5,0.980348,0.551333,0.940859,0.773164,0.884038,0.906160,0.827166,0.984818
267,Luka Dončić,7.118835,6.988819,0.130016,[PG],5,0.963816,0.651968,0.948873,0.410095,0.988250,0.894732,0.679031,0.991542
21,Anthony Edwards,7.047873,6.898323,0.199400,[SG],4,0.957162,0.428426,0.837249,0.554548,0.881225,0.972129,0.820035,0.969518
394,Victor Wembanyama,6.963580,6.963580,0.001720,[C],1,0.786420,0.566667,0.977778,0.570370,0.814815,0.949383,1.000000,0.933333
240,Kevin Durant,6.875848,6.589359,0.381985,[PF],4,0.759418,0.838335,0.782641,0.917880,0.856964,0.555653,0.904143,0.926216
262,LeBron James,6.782174,6.755843,0.026331,"[PF, C, PG]",5,0.848094,0.800188,0.890970,0.357525,0.956154,0.820805,0.758271,0.951346
358,Shai Gilgeous-Alexander,6.774399,6.486502,0.287897,"[PG, SG]",5,0.638560,0.716288,0.714692,0.744785,0.889023,0.820311,0.832299,0.930540
230,Kawhi Leonard,6.767599,6.790008,-0.029879,[SF],4,0.786050,0.764092,0.826440,0.908814,0.824716,0.943039,0.718594,0.927358
289,Mikal Bridges,6.730481,6.567458,0.163023,"[SF, SG]",5,0.834935,0.686305,0.775311,0.765664,0.766881,0.947903,0.818130,0.849158


In [13]:
# new_df.to_csv("/Users/michaelbinger/Documents/Projects/Fantasy-Baseball-Analysis/Fantasy_Basketball/nba_trends.csv")
# final_yahoo_nba_df.to_csv("/Users/michaelbinger/Documents/Projects/Fantasy-Baseball-Analysis/Fantasy_Basketball/nba_stats.csv")


In [14]:
new_pos_list = []
pos_list = new_df['Pos'].tolist()
for item in pos_list:
    new_string = []
    for pos in item:
        string = str(pos)
        new_string = f'{new_string},{string}'
    new_pos_list.append(new_string)

final_pos_list = []
for i in new_pos_list:
    i = i.replace('[],', '')
    final_pos_list.append(i)
    
final_pos_list

new_df.drop('Pos', axis = 1, inplace = True)
new_df['Pos'] = final_pos_list

new_df

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
309,Nikola Jokić,7.229185,7.225615,0.003570,5,0.641431,0.905583,0.992384,0.702533,0.992733,0.960128,0.877943,0.978469,C
190,Jayson Tatum,7.200213,7.170453,0.029761,5,0.980348,0.551333,0.940859,0.773164,0.884038,0.906160,0.827166,0.984818,"PF,SF"
267,Luka Dončić,7.118835,6.988819,0.130016,5,0.963816,0.651968,0.948873,0.410095,0.988250,0.894732,0.679031,0.991542,PG
21,Anthony Edwards,7.047873,6.898323,0.199400,4,0.957162,0.428426,0.837249,0.554548,0.881225,0.972129,0.820035,0.969518,SG
394,Victor Wembanyama,6.963580,6.963580,0.001720,1,0.786420,0.566667,0.977778,0.570370,0.814815,0.949383,1.000000,0.933333,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,Juwan Morgan,0.501235,0.501235,0.000124,1,0.111111,0.590123,0.014815,0.002469,0.017284,0.018519,0.038272,0.002469,PF
72,Colby Jones,0.409259,0.409259,0.000101,1,0.092593,0.100000,0.027160,0.029630,0.050617,0.032099,0.102469,0.019753,SG
637,Malcolm Miller,0.325858,0.325858,0.000081,1,0.149077,0.241425,0.002639,0.006596,0.007916,0.011873,0.025066,0.002639,SF
525,Joe Wieskamp,0.302885,0.302885,0.000075,1,0.175481,0.033654,0.004808,0.036058,0.016827,0.012019,0.044471,0.007212,SG


In [15]:
draft_df = new_df

In [16]:
draft_df.head(50)

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
309,Nikola Jokić,7.229185,7.225615,0.003570,5,0.641431,0.905583,0.992384,0.702533,0.992733,0.960128,0.877943,0.978469,C
190,Jayson Tatum,7.200213,7.170453,0.029761,5,0.980348,0.551333,0.940859,0.773164,0.884038,0.906160,0.827166,0.984818,"PF,SF"
267,Luka Dončić,7.118835,6.988819,0.130016,5,0.963816,0.651968,0.948873,0.410095,0.988250,0.894732,0.679031,0.991542,PG
21,Anthony Edwards,7.047873,6.898323,0.199400,4,0.957162,0.428426,0.837249,0.554548,0.881225,0.972129,0.820035,0.969518,SG
394,Victor Wembanyama,6.963580,6.963580,0.001720,1,0.786420,0.566667,0.977778,0.570370,0.814815,0.949383,1.000000,0.933333,C
240,Kevin Durant,6.875848,6.589359,0.381985,4,0.759418,0.838335,0.782641,0.917880,0.856964,0.555653,0.904143,0.926216,PF
262,LeBron James,6.782174,6.755843,0.026331,5,0.848094,0.800188,0.890970,0.357525,0.956154,0.820805,0.758271,0.951346,"PF,C,PG"
358,Shai Gilgeous-Alexander,6.774399,6.486502,0.287897,5,0.638560,0.716288,0.714692,0.744785,0.889023,0.820311,0.832299,0.930540,"PG,SG"
230,Kawhi Leonard,6.767599,6.790008,-0.029879,4,0.786050,0.764092,0.826440,0.908814,0.824716,0.943039,0.718594,0.927358,SF
289,Mikal Bridges,6.730481,6.567458,0.163023,5,0.834935,0.686305,0.775311,0.765664,0.766881,0.947903,0.818130,0.849158,"SF,SG"


In [17]:
######################################################################################################################
######################################################################################################################
######################################################################################################################
#### DRAFT DAY FUNCTIONS
   
# DROP A PLAYER 
def drafted(player):
    global draft_df
    draft_df = draft_df[draft_df.Player != player]
    return draft_df.head(25)
    
# FILTER PLAYERS BY POSITION
def position_filter(Pos):
    filtered_draft_df = draft_df[draft_df['Pos'].str.contains(Pos)]
    return filtered_draft_df.head(25)

# PULL STAT CATEGORY LEADERS
def stat_leaders(CAT):
    global final_yahoo_nba_df
    final_yahoo_nba_df = final_yahoo_nba_df.sort_values([CAT], ascending=[False])
    return final_yahoo_nba_df.head(25)

In [18]:
# draft_df = draft_df.sort_values(['Weighted Rank'], ascending=[False])
# draft_df.head(50)

In [19]:
drafted('')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
309,Nikola Jokić,7.229185,7.225615,0.003570,5,0.641431,0.905583,0.992384,0.702533,0.992733,0.960128,0.877943,0.978469,C
190,Jayson Tatum,7.200213,7.170453,0.029761,5,0.980348,0.551333,0.940859,0.773164,0.884038,0.906160,0.827166,0.984818,"PF,SF"
267,Luka Dončić,7.118835,6.988819,0.130016,5,0.963816,0.651968,0.948873,0.410095,0.988250,0.894732,0.679031,0.991542,PG
21,Anthony Edwards,7.047873,6.898323,0.199400,4,0.957162,0.428426,0.837249,0.554548,0.881225,0.972129,0.820035,0.969518,SG
394,Victor Wembanyama,6.963580,6.963580,0.001720,1,0.786420,0.566667,0.977778,0.570370,0.814815,0.949383,1.000000,0.933333,C
240,Kevin Durant,6.875848,6.589359,0.381985,4,0.759418,0.838335,0.782641,0.917880,0.856964,0.555653,0.904143,0.926216,PF
262,LeBron James,6.782174,6.755843,0.026331,5,0.848094,0.800188,0.890970,0.357525,0.956154,0.820805,0.758271,0.951346,"PF,C,PG"
358,Shai Gilgeous-Alexander,6.774399,6.486502,0.287897,5,0.638560,0.716288,0.714692,0.744785,0.889023,0.820311,0.832299,0.930540,"PG,SG"
230,Kawhi Leonard,6.767599,6.790008,-0.029879,4,0.786050,0.764092,0.826440,0.908814,0.824716,0.943039,0.718594,0.927358,SF
289,Mikal Bridges,6.730481,6.567458,0.163023,5,0.834935,0.686305,0.775311,0.765664,0.766881,0.947903,0.818130,0.849158,"SF,SG"
